In [5]:
import requests
import os
import sys

# --- CONFIGURATION ---
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3"
WORKSPACE_DIR = "./contest_workspace"

os.makedirs(WORKSPACE_DIR, exist_ok=True)

# --- LLM CLIENT ---
def call_llm(system_prompt, user_prompt):
    full_prompt = f"System: {system_prompt}\nUser: {user_prompt}"
    payload = {
        "model": MODEL_NAME, "prompt": full_prompt, "stream": False,
        "options": {"temperature": 0.2, "num_ctx": 8192}
    }
    try:
        print("   (Copilot is thinking...)", end="\r")
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        return response.json()['response']
    except Exception as e:
        return f"LLM_ERROR: {str(e)}"

def save_code(code_text, filename):
    clean_code = code_text
    if "```python" in code_text:
        clean_code = code_text.split("```python")[1].split("```")[0]
    elif "```" in code_text:
        clean_code = code_text.split("```")[1].split("```")[0]
    clean_code = clean_code.strip()
    
    filepath = os.path.join(WORKSPACE_DIR, filename)
    with open(filepath, "w") as f:
        f.write(clean_code)
    return filepath, clean_code

# --- UPDATED AGENTS ---

class DataCopilot:
    def __init__(self, specs):
        self.role = "Data Engineer"
        # UPDATED: Now strictly prompted with Data Description
        self.system = (
            f"You are an Autonomous Data Engineer. Contest: '{specs['name']}'.\n"
            f"ROOT SEARCH PATH: '{os.path.abspath(ALL_DATA_PATH)}'.\n"
            f"DATA DESCRIPTION: {specs['data_description']}.\n\n"
            
            "TASK: Write a Python script to Find, Inspect, and Prepare data.\n"
            "LOGIC FLOW:\n"
            "1. SEARCH: Find the subfolder inside ROOT SEARCH PATH that matches the Contest Name.\n"
            "2. INSPECT: List all files in that folder.\n"
            "3. DECIDE STRATEGY:\n"
            "   - CASE A: If you find separate 'train' and 'test' files -> Load them directly.\n"
            "   - CASE B: If you find 'train' and 'test' and 'val' -> Merge 'val' into 'train' or ignore it, then Load train/test.\n"
            "   - CASE C: check in folder, if only one file csv => load this as data files.\n"
            "4. PREPARE: Handle missing values and Encode categoricals (OneHot/Label).\n"
            "5. DETECT: Check label and data, label can based on data_description, rename this col into 'target' "
            "6. OUTPUT: Save 'processed_train.csv' and 'processed_test.csv' to the current directory.\n"
            "7. PRINT: 'DATA_PROCESSED: processed_train.csv processed_test.csv' at the end.\n"
            "OUTPUT: ONLY valid Python code."
        )

    def suggest(self, feedback=""):
        prompt = "Generate the data preparation script based on the Data Description."
        if feedback:
            prompt += f"\nCONTEXT: The user ran previous code and reported: {feedback}"
        return call_llm(self.system, prompt)

class ModelCopilot:
    def __init__(self, specs):
        self.specs = specs 
        self.role = "Modeling Copilot"
        self.system = (
            f"You are a ML Modeling Copilot. Target Metric: {specs['target_metric']}. "
            f"Output Specs: {specs['output_expectations']}. "
            "Task: Write Python code to load the PROCESSED data, train a model, and print validation metrics. "
            "Output Format: \n"
            "1. A brief note on strategy.\n"
            "2. The valid Python code block."
        )

    def suggest(self, feedback=""):
        prompt = f"Generate training code optimizing for {self.specs['target_metric']}."
        if feedback:
            prompt += f"\nCONTEXT: User reported previous result/error: {feedback}"
        return call_llm(self.system, prompt)


# --- MANAGER ---

class CopilotManager:
    def __init__(self, specs):
        self.specs = specs
        self.data_agent = DataCopilot(specs)
        self.model_agent = ModelCopilot(self.specs)

    def present_solution(self, agent_name, raw_response, filename):
        filepath, clean_code = save_code(raw_response, filename)
        print(f"\n{'='*60}")
        print(f"🤖 {agent_name} suggests the following:")
        print(f"📂 Saved to: {filepath}")
        print(f"{'-'*60}")
        
        # Preview lines
        lines = clean_code.splitlines()
        if len(lines) > 20:
            print("\n".join(lines[:10]))
            print(f"... [ {len(lines)-15} lines hidden ] ...")
            print("\n".join(lines[-5:]))
        else:
            print(clean_code)
        
        print(f"{'='*60}")
        print(f"\n👉 ACTION: Run 'python {filepath}' in your terminal.")
        return filepath

    def start_session(self):
        print(f"🎓 ACADEMIMI COPILOT INITIALIZED: {self.specs['name']}")
        
        # PHASE 1: DATA
        print("\n--- PHASE 1: DATA PREPARATION ---")
        feedback = ""
        while True:
            response = self.data_agent.suggest(feedback)
            self.present_solution("Data Agent", response, "01_data_prep.py")
            
            print("\nDid the code run successfully? (y/n/error)")
            user_status = input(">> ").strip()
            
            if user_status.lower() == 'y':
                print("✅ Data Phase Complete.")
                break
            else:
                feedback = f"Error or User Feedback: {user_status}"

        # PHASE 2: MODELING
        print("\n--- PHASE 2: MODELING ---")
        feedback = ""
        while True:
            response = self.model_agent.suggest(feedback)
            self.present_solution("Model Agent", response, "02_train_model.py")
            
            print("\nDid the code produce a good score? (y/n/error)")
            user_status = input(">> ").strip()
            
            if user_status.lower() == 'y':
                print("🎉 Workflow Complete.")
                break
            else:
                feedback = user_status

if __name__ == "__main__":
    # --- HERE IS WHERE YOU DEFINE THE DATA DESCRIPTION ---
    my_challenge = {
        "name": "Titanic Survival",
        
        "requirements": "Predict survival (0/1). Train on train.csv, predict on test.csv.",
        
        # NEW FIELD: The Agent will read this to decide how to code
        "data_description": """
        The dataset contains the following columns:
        - PassengerId: Unique ID (Drop this for training)
        - Survived: Target variable (0 = No, 1 = Yes)
        - Pclass: Ticket class (1, 2, 3). Treat as Ordinal.
        - Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')
        - Sex: 'male'/'female'. (Map to 0/1)
        - Age: Numeric. Contains missing values (Fill with Median by Title).
        - Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')
        - Embarked: Port of Embarkation (C, Q, S). (One-Hot Encode).
        """,
        
        "target_metric": "Accuracy",
        "output_expectations": "submission.csv with PassengerId, Survived"
    }

    manager = CopilotManager(my_challenge)
    manager.start_session()

🎓 ACADEMIMI COPILOT INITIALIZED: Titanic Survival

--- PHASE 1: DATA PREPARATION ---
   (Copilot is thinking...)
🤖 Data Agent suggests the following:
📂 Saved to: ./contest_workspace/01_data_prep.py
------------------------------------------------------------
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Set root search path
root_search_path = '/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets'

# Find subfolder matching contest name
contest_folder = [f for f in os.listdir(root_search_path) if f.startswith('Titanic Survival')][0]
contest_folder_path = os.path.join(root_search_path, contest_folder)
... [ 31 lines hidden ] ...
# Output prepared data
df.to_csv('processed_train.csv', index=False)
test_df.to_csv('processed_test.csv', index=False)

print("DATA_PROCESSED: processed_train.csv processed_test.csv")

👉 ACTION: Run 'python ./contest_workspace/01_data_prep.py' in your terminal.

Did the code run successfully? (y/n/error)
✅ Data Pha

In [1]:
import requests
import os
import sys
import subprocess  # <--- FIXED: Added missing import

# --- CONFIGURATION ---
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3"
WORKSPACE_DIR = "./contest_workspace"
PYTHON_EXEC = sys.executable # <--- FIXED: Define python executable

# 1. ROOT DATA FOLDER (Agent searches inside here)
ALL_DATA_PATH = "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets" 

# Create directories
os.makedirs(WORKSPACE_DIR, exist_ok=True)
os.makedirs(ALL_DATA_PATH, exist_ok=True)

# --- LLM CLIENT ---
def call_llm(system_prompt, user_prompt):
    full_prompt = f"System: {system_prompt}\nUser: {user_prompt}"
    payload = {
        "model": MODEL_NAME, "prompt": full_prompt, "stream": False,
        "options": {"temperature": 0.2, "num_ctx": 8192}
    }
    try:
        print("   (Copilot is thinking...)", end="\r")
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        return response.json().get('response', '')
    except Exception as e:
        return f"LLM_ERROR: {str(e)}"

def save_code(code_text, filename):
    clean_code = code_text
    # Robust extraction of code blocks
    if "```python" in code_text:
        clean_code = code_text.split("```python")[1].split("```")[0]
    elif "```" in code_text:
        clean_code = code_text.split("```")[1].split("```")[0]
    
    clean_code = clean_code.strip()

    # Remove accidental "python" string at start
    lines = clean_code.splitlines()
    if lines and lines[0].strip().lower() == "python":
        clean_code = "\n".join(lines[1:]).strip()

    filepath = os.path.join(WORKSPACE_DIR, filename)
    with open(filepath, "w") as f:
        f.write(clean_code)
    return filepath, clean_code


def execute_script(filepath):
    print(f"⚡ Executing {filepath}...")
    
    # Run inside the workspace directory
    filename = os.path.basename(filepath)
    
    try:
        result = subprocess.run(
            [PYTHON_EXEC, filename], 
            cwd=WORKSPACE_DIR,       
            capture_output=True,
            text=True,
            timeout=120 
        )
        return result
    except subprocess.TimeoutExpired:
        print("   ❌ Timeout Expired!")
        return None

# --- UPDATED AGENTS ---

class DataCopilot:
    def __init__(self, specs):
        self.role = "Data Engineer"
        self.specs = specs
        self.system = (
            f"You are an Autonomous Data Engineer. Contest: '{specs['name']}'.\n"
            f"ROOT SEARCH PATH: '{os.path.abspath(ALL_DATA_PATH)}'.\n"
            f"DATA DESCRIPTION: {specs['data_description']}.\n\n"
            
            "TASK: Write a Python script to Find, Inspect, and Prepare data.\n"
            "LOGIC FLOW:\n"
            "1. SEARCH: Find the subfolder inside ROOT SEARCH PATH that matches the Contest Name.\n"
            "2. INSPECT: List all files in that folder.\n"
            "3. DECIDE STRATEGY:\n"
            "   - CASE A: If you find separate 'train' and 'test' files -> Load them directly.\n"
            "   - CASE B: If you find 'train' and 'test' and 'val' -> Merge 'val' into 'train'.\n"
            "   - CASE C: If only one csv file exists -> Load it and split 80/20.\n"
            "4. PREPARE: Handle missing values and Encode categoricals (OneHot/Label).\n"
            "5. OUTPUT: Save 'processed_train.csv' and 'processed_test.csv' to the current directory.\n"
            "6. PRINT: 'DATA_PROCESSED: processed_train.csv processed_test.csv' at the very end.\n"
            "OUTPUT: ONLY valid Python code block."
        )

    def generate(self, feedback=""):
        # Base Prompt
        prompt = f"Generate the code for {self.role}."
        
        # Inject Feedback if it exists
        if feedback:
            prompt += (
                "\n\n⚠️ CRITICAL FIX REQUIRED ⚠️\n"
                f"{feedback}\n"
                "Review the error above. REWRITE the code to fix this specific issue."
            )
            
        return call_llm(self.system, prompt)

class ModelCopilot:
    def __init__(self, specs):
        self.specs = specs 
        self.role = "Modeling Copilot"
        self.train_file = "processed_train.csv"
        self.test_file = "processed_test.csv"
        self._update_system_prompt()

    # FIXED: Added helper to update prompt dynamically
    def _update_system_prompt(self):
        self.system = (
            f"You are a ML Modeling Copilot. Target Metric: {self.specs['target_metric']}. "
            f"Output Specs: {self.specs['output_expectations']}. "
            f"TRAIN DATA: {self.train_file}, TEST DATA: {self.test_file}\n"
            "Task: Write Python code to load the PROCESSED data, train a model, and print validation metrics. "
            "Output Format: \n"
            "1. Load data from the files specified above.\n"
            "2. Train model (RandomForest/XGBoost).\n"
            "3. Create submission file.\n"
            "4. PRINT 'FINAL_METRIC: <score>' at the end.\n"
            "OUTPUT: ONLY valid Python code."
        )

    # FIXED: Added method to receive file names from Manager
    def set_data_files(self, train_f, test_f):
        self.train_file = train_f
        self.test_file = test_f
        self._update_system_prompt()

    def generate(self, feedback=""):
        # Base Prompt
        prompt = f"Generate the code for {self.role}."
        
        # Inject Feedback if it exists
        if feedback:
            prompt += (
                "\n\n⚠️ CRITICAL FIX REQUIRED ⚠️\n"
                f"{feedback}\n"
                "Review the error above. REWRITE the code to fix this specific issue."
            )
            
        return call_llm(self.system, prompt)

# --- MANAGER ---

class CopilotManager:
    def __init__(self, specs):
        self.specs = specs
        self.data_agent = DataCopilot(specs)
        self.model_agent = ModelCopilot(self.specs)

    def run_phase(self, phase_name, agent, filename, max_retries=3):
        print(f"\n🔵 PHASE: {phase_name}")
        feedback = ""  # Start with no errors
        
        for attempt in range(max_retries):
            print(f"   Attempt {attempt+1}...")
            
            # 1. Generate Code (Feedback is empty on first try)
            raw_response = agent.generate(feedback)
            filepath, _ = save_code(raw_response, filename)
            
            # 2. Execute
            result = execute_script(filepath)
            
            # 3. Validation Logic
            if result and result.returncode == 0:
                print(f"   ✅ Success.")
                return True, result.stdout
            
            # 4. ERROR HANDLING (Self-Correction)
            else:
                print(f"   ❌ Failed.")
                
                # Capture the specific error
                if result:
                    # Get the last 20 lines of the traceback (most useful part)
                    error_log = result.stderr.strip()
                    tail_error = "\n".join(error_log.splitlines()[-20:])
                    print(f"      Err: {tail_error}")
                    
                    # UPDATE FEEDBACK FOR NEXT LOOP
                    feedback = (
                        f"Previous code crashed with this error:\n"
                        f"```text\n{tail_error}\n```\n"
                        "Fix the code to handle this error. Do not make the same mistake."
                    )
                else:
                    feedback = "The code timed out after 120 seconds. Optimize it or check for infinite loops."
        
        return False, "Failed after max retries"

    def execute_script(filepath):
        print(f"⚡ Executing {filepath}...")
        filename = os.path.basename(filepath)
        
        try:
            # CAPTURE both stdout (normal output) and stderr (crash errors)
            result = subprocess.run(
                [PYTHON_EXEC, filename], 
                cwd=WORKSPACE_DIR,       
                capture_output=True,     # <--- Critical: Captures the output
                text=True,               # <--- Critical: Decodes to string
                timeout=120 
            )
            return result
        except subprocess.TimeoutExpired:
            print("   ❌ Timeout Expired!")
            return None

    def start(self):
        print(f"🚀 STARTING JOB: {self.specs['name']}")
        
        # Phase 1: Smart Data Search
        success, output = self.run_phase("Data Prep (Smart Search)", self.data_agent, "01_data_prep.py")
        if not success: 
            print("❌ Data Phase Failed. Stopping.")
            return

        # Parse Data Files from Output
        for line in output.splitlines():
            if "DATA_PROCESSED:" in line:
                try:
                    parts = line.split("DATA_PROCESSED:")[1].strip().split()
                    if len(parts) >= 2:
                        self.model_agent.set_data_files(parts[0], parts[1])
                        print(f"   📂 Identified Data Files: {parts[0]}, {parts[1]}")
                except Exception as e:
                    print(f"   ⚠️ Parsing warning: {e}")

        # Phase 2: Training
        success, output = self.run_phase("Model Training", self.model_agent, "02_train_model.py")
        if not success: return

        # Result Parsing
        for line in output.splitlines():
            if "FINAL_METRIC" in line:
                print(f"\n🏆 {line.strip()}")
                break

if __name__ == "__main__":

    # --- RUN AGENT ---
    my_challenge = {
        "name": "Titanic",
        "requirements": "Predict survival (0/1). Train on train.csv, predict on test.csv.",
        
        "data_description": """
        The dataset contains the following columns:
        - PassengerId: Unique ID (Drop this for training)
        - Survived: Target variable (0 = No, 1 = Yes)
        - Pclass: Ticket class (1, 2, 3). Treat as Ordinal.
        - Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')
        - Sex: 'male'/'female'. (Map to 0/1)
        - Age: Numeric. Contains missing values (Fill with Median by Title).
        - Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')
        - Embarked: Port of Embarkation (C, Q, S). (One-Hot Encode).
        """,
        
        "target_metric": "Accuracy",
        "output_expectations": "submission.csv with PassengerId, Survived"
    }
    
    manager = CopilotManager(my_challenge)
    manager.start()

🚀 STARTING JOB: Titanic

🔵 PHASE: Data Prep (Smart Search)
   Attempt 1...
⚡ Executing ./contest_workspace/01_data_prep.py...
   ❌ Failed.
      Err: Traceback (most recent call last):
  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/contest_workspace/01_data_prep.py", line 29, in <module>
    train_df, test_df = df.sample(frac=0.8, random_state=42), df.drop(train_df.index)
NameError: name 'train_df' is not defined
   Attempt 2...
⚡ Executing ./contest_workspace/01_data_prep.py...
   ❌ Failed.
      Err: Traceback (most recent call last):
  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/contest_workspace/01_data_prep.py", line 37, in <module>
    train_df['Title'] = train_df.Name.str.extract('([A-Za-z]+)', expand=False)
NameError: name 'train_df' is not defined
   Attempt 3...
⚡ Executing ./contest_workspace/01_data_prep.py...
   ❌ Failed.
      Err: Traceback (most recent call last):
  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/contest_wor

In [ ]:
import requests
import os
import sys
import subprocess
import time

# --- CONFIGURATION ---
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3"  # Ensure you have this model: `ollama pull llama3`
WORKSPACE_DIR = "./contest_workspace"
PYTHON_EXEC = sys.executable 

# ROOT DATA PATH (Adjust this to your actual path)
ALL_DATA_PATH = "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets" 

# Setup Directories
os.makedirs(WORKSPACE_DIR, exist_ok=True)
os.makedirs(ALL_DATA_PATH, exist_ok=True)


# --- 1. LLM COMMUNICATION LAYER ---
def call_llm(system_prompt, user_prompt):
    """Sends request to Ollama with a strict JSON payload."""
    full_prompt = f"System: {system_prompt}\nUser: {user_prompt}"
    payload = {
        "model": MODEL_NAME, 
        "prompt": full_prompt, 
        "stream": False,
        "options": {
            "temperature": 0.2, # Low temp for precise coding
            "num_ctx": 16384,    # Large context for code + error logs
            "stop": ["User:", "System:", "```python\n\n"] # Stop tokens
        }
    }
    try:
        print("   🧠 (Agent is thinking...)", end="\r")
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        return response.json().get('response', '')
    except Exception as e:
        return f"LLM_ERROR: {str(e)}"

def save_code_to_file(code_text, filename):
    """Extracts Python block and saves to file."""
    clean_code = code_text
    # Extract code between markdown ticks
    if "```python" in code_text:
        clean_code = code_text.split("```python")[1].split("```")[0]
    elif "```" in code_text:
        clean_code = code_text.split("```")[1].split("```")[0]
    
    clean_code = clean_code.strip()
    
    # Remove accidental starting text
    lines = clean_code.splitlines()
    if lines and lines[0].lower().startswith("python"):
        clean_code = "\n".join(lines[1:])

    filepath = os.path.join(WORKSPACE_DIR, filename)
    with open(filepath, "w") as f:
        f.write(clean_code)
    return filepath, clean_code

def execute_code(filepath):
    """Executes the script and returns (Success_Bool, Output/Error)."""
    print(f"   ⚡ Executing {os.path.basename(filepath)}...")
    try:
        result = subprocess.run(
            [PYTHON_EXEC, os.path.basename(filepath)], 
            cwd=WORKSPACE_DIR,       
            capture_output=True,     
            text=True,               
            timeout=120 
        )
        if result.returncode == 0:
            return True, result.stdout
        else:
            return False, result.stderr
    except subprocess.TimeoutExpired:
        return False, "TIMEOUT: Script took longer than 120 seconds."
    except Exception as e:
        return False, str(e)


# --- 2. AGENT DEFINITIONS ---

class BaseAgent:
    """Parent class capable of Generating and Fixing code."""
    def __init__(self, role, system_instruction):
        self.role = role
        self.system = system_instruction

    def generate_initial_code(self):
        """Step 1: Create code from scratch."""
        prompt = f"Write the Python code for the {self.role} task based on your System Instructions."
        return call_llm(self.system, prompt)

    def fix_code(self, broken_code, error_log):
        """Step 2: Self-Reflection & Fix."""
        prompt = f"""
        ### SELF-REFLECTION & FIX TASK
        Your previous code failed to execute.
        
        **1. THE BROKEN CODE:**
        ```python
        {broken_code}
        ```
        
        **2. THE ERROR LOG:**
        {error_log}
        
        **INSTRUCTION:**
        - Analyze the error log carefully.
        - Rewrite the code to fix the specific bug.
        - Return the COMPLETE corrected Python script.
        """
        return call_llm(self.system, prompt)


class DataAgent(BaseAgent):
    def __init__(self, specs):
        system = (
            f"You are an Expert Data Engineer. Contest: '{specs['name']}'.\n"
            f"ROOT PATH: '{os.path.abspath(ALL_DATA_PATH)}'.\n"
            f"DATA SPECS: {specs['data_description']}.\n\n"
            "TASK: Write a robust Python script to Find, Load, and Preprocess data.\n"
            "STRICT RULES:\n"
            "1. USE `os.walk` to find the folder matching the Contest Name.\n"
            "2. CHECK file count using `glob`. IF 1 file -> Split 80/20. IF 2 files -> Load separately.\n"
            "3. PREPROCESS: Handle missing values and categorical encoding.\n"
            "4. OUTPUT: Save 'processed_train.csv' and 'processed_test.csv'.\n"
            "5. PRINT 'DATA_READY: processed_train.csv processed_test.csv' at the very end.\n"
            "OUTPUT FORMAT: Single valid ```python block."
        )
        super().__init__("Data Engineering", system)


class ModelAgent(BaseAgent):
    def __init__(self, specs):
        self.specs = specs
        # Placeholder system prompt, will be updated when data is ready
        self.train_file = "processed_train.csv"
        self.test_file = "processed_test.csv"
        self._update_system()

    def _update_system(self):
        system = (
            f"You are a Kaggle Grandmaster. Goal: Maximize {self.specs['target_metric']}.\n"
            f"INPUT FILES: {self.train_file}, {self.test_file}\n"
            "TASK: Write Python code to Train a Model and Generate Submission.\n"
            "STEPS:\n"
            "1. Load the processed CSV files.\n"
            "2. Train a robust model (RandomForest/XGBoost/LightGBM).\n"
            "3. Evaluate on internal validation set.\n"
            "4. Generate 'submission.csv'.\n"
            "5. PRINT 'FINAL_SCORE: <score>' at the end.\n"
            "OUTPUT FORMAT: Single valid ```python block."
        )
        super().__init__("Model Training", system)
    
    def set_data_files(self, train, test):
        self.train_file = train
        self.test_file = test
        self._update_system()


# --- 3. THE MANAGER (WORKFLOW ORCHESTRATOR) ---

class CopilotManager:
    def __init__(self, specs):
        self.specs = specs
        self.data_agent = DataAgent(specs)
        self.model_agent = ModelAgent(specs)

    def run_agent_cycle(self, agent, filename, max_retries=3):
        """
        The Core Loop: Gen -> Exec -> Fix -> Retry
        """
        print(f"\n🔹 STARTED AGENT: {agent.role}")
        
        current_code = ""
        last_error = ""

        for attempt in range(max_retries + 1):
            # --- STEP A: GENERATE / FIX ---
            if attempt == 0:
                print("   📝 Generatng initial code...")
                raw_response = agent.generate_initial_code()
            else:
                print(f"   🔧 Fixing bug (Attempt {attempt}/{max_retries})...")
                # Feedback Loop: Send broken code + error back to agent
                raw_response = agent.fix_code(current_code, last_error)

            # --- STEP B: SAVE & PREPARE ---
            filepath, clean_code = save_code_to_file(raw_response, filename)
            current_code = clean_code # Update memory

            # --- STEP C: EXECUTE ---
            success, output = execute_code(filepath)

            # --- STEP D: EVALUATE ---
            if success:
                print("   ✅ Execution Successful.")
                return True, output
            else:
                print("   ❌ Execution Failed.")
                # Capture error for the next 'Fix' loop
                last_error = output.strip()
                # Print short preview of error
                error_preview = "\n".join(last_error.splitlines()[-10:])
                print(f"      [Bug Log]: {error_preview}")

        print("   💀 Max retries reached. Moving on...")
        return False, last_error

    def start_workflow(self):
        print(f"🚀 STARTING WORKFLOW: {self.specs['name']}")
        start_time = time.time()

        # ==========================================
        # 1. DATA AGENT PHASE
        # ==========================================
        success, output = self.run_agent_cycle(self.data_agent, "01_data_pipeline.py")
        
        if not success:
            print("🛑 Data Agent failed. Workflow stopped.")
            return

        # Parse Output to find file names
        train_file, test_file = None, None
        for line in output.splitlines():
            if "DATA_READY:" in line:
                try:
                    parts = line.split("DATA_READY:")[1].strip().split()
                    train_file, test_file = parts[0], parts[1]
                    print(f"   📂 Passed to Model Agent: {train_file}, {test_file}")
                except:
                    print("   ⚠️ Could not parse file names, using defaults.")

        # ==========================================
        # 2. MODEL AGENT PHASE
        # ==========================================
        if train_file and test_file:
            self.model_agent.set_data_files(train_file, test_file)

        success, output = self.run_agent_cycle(self.model_agent, "02_model_pipeline.py")

        if success:
            for line in output.splitlines():
                if "FINAL_SCORE:" in line:
                    print(f"\n🏆 {line.strip()}")
        
        print(f"\n🏁 Workflow Finished in {time.time() - start_time:.2f}s")


# --- 4. EXECUTION ---
if __name__ == "__main__":
    
    # Requirement Spec
    my_challenge = {
        "name": "Titanic",
        "requirements": "Predict survival (0/1). Train on train.csv, predict on test.csv.",
        
        "data_description": """
        The dataset contains the following columns:
        - PassengerId: Unique ID (Drop this for training)
        - Survived: Target variable (0 = No, 1 = Yes)
        - Pclass: Ticket class (1, 2, 3). Treat as Ordinal.
        - Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')
        - Sex: 'male'/'female'. (Map to 0/1)
        - Age: Numeric. Contains missing values (Fill with Median by Title).
        - Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')
        - Embarked: Port of Embarkation (C, Q, S). (One-Hot Encode).
        """,
        
        "target_metric": "Accuracy",
        "output_expectations": "submission.csv with PassengerId, Survived"
    }

    # Run
    manager = CopilotManager(my_challenge)
    manager.start_workflow()

🚀 STARTING WORKFLOW: Titanic

🔹 STARTED AGENT: Data Engineering
   📝 Generatng initial code...
   ⚡ Executing 01_data_pipeline.py...
   ❌ Execution Failed.
      [Bug Log]: Traceback (most recent call last):
  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/contest_workspace/01_data_pipeline.py", line 33, in <module>
    test_df['Title'] = test_df['Name'].apply(extract_title) if test_file else None
TypeError: 'NoneType' object does not support item assignment
   🔧 Fixing bug (Attempt 1/3)...
   ⚡ Executing 01_data_pipeline.py...
   ❌ Execution Failed.
      [Bug Log]: Traceback (most recent call last):
  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/contest_workspace/01_data_pipeline.py", line 32, in <module>
    test_df['Sex'] = sex_encoder.transform(test_df['Sex']) if test_file else None
TypeError: 'NoneType' object does not support item assignment
   🔧 Fixing bug (Attempt 2/3)...
   ⚡ Executing 01_data_pipeline.py...
   ❌ Execution Failed.
      [Bug Log]

In [ ]:
import requests
import os
import sys
import subprocess
import time

# --- CONFIGURATION ---
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3"
WORKSPACE_DIR = "./contest_workspace"
PYTHON_EXEC = sys.executable

# 1. ROOT DATA FOLDER (Agent searches inside here)
ALL_DATA_PATH = "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets" 

os.makedirs(WORKSPACE_DIR, exist_ok=True)
os.makedirs(ALL_DATA_PATH, exist_ok=True)

# # --- UTILS ---
# def call_llm(system_prompt, user_prompt):
#     full_prompt = f"System: {system_prompt}\nUser: {user_prompt}"
#     payload = {
#         "model": MODEL_NAME, "prompt": full_prompt, "stream": False,
#         "options": {"temperature": 0.2, "num_ctx": 8192}
#     }
#     try:
#         print("   (Agent is thinking...)", end="\r")
#         response = requests.post(OLLAMA_URL, json=payload)
#         response.raise_for_status()
#         return response.json()['response']
#     except Exception as e:
#         return f"LLM_ERROR: {str(e)}"

# --- UTILS ---
def call_llm(message):
    payload = {
        "model": MODEL_NAME, "prompt": message, "stream": False,
        "options": {"temperature": 0.2, "num_ctx": 8192}
    }
    try:
        print("   (Agent is thinking...)", end="\r")
        response = requests.post(OLLAMA_URL, json=payload)
        response.raise_for_status()
        return response.json()['response']
    except Exception as e:
        return f"LLM_ERROR: {str(e)}"

def save_code(code_text, filename):
    clean_code = code_text
    if "```python" in code_text:
        clean_code = code_text.split("```python")[1].split("```")[0]
    elif "```" in code_text:
        clean_code = code_text.split("```")[1].split("```")[0]
    clean_code = clean_code.strip()

    lines = clean_code.splitlines()
    if lines and lines[0].strip().lower() == "python":
        clean_code = "\n".join(lines[1:]).strip()

    filepath = os.path.join(WORKSPACE_DIR, filename)
    with open(filepath, "w") as f:
        f.write(clean_code)
    return filepath, clean_code


def execute_script(filepath):
    print(f"⚡ Executing {filepath}...")
    
    # FIX: Extract just the filename ('01_data_prep.py') 
    # because cwd is already set to WORKSPACE_DIR
    filename = os.path.basename(filepath)
    
    try:
        result = subprocess.run(
            [PYTHON_EXEC, filename], # <--- Changed from 'filepath' to 'filename'
            cwd=WORKSPACE_DIR,       # Execution happens inside the workspace folder
            capture_output=True,
            text=True,
            timeout=120 
        )
        return result
    except subprocess.TimeoutExpired:
        return None


# --- AGENTS ---

class DataAgent:
    def __init__(self, specs):
        self.money = {}
        self.specs = specs

        self.system = (
            f"You are an Autonomous Data Engineer. Contest: '{specs['name']}'.\n"
            f"ROOT SEARCH PATH: '{os.path.abspath(ALL_DATA_PATH)}'.\n"
            f"DATA DESCRIPTION: {specs['data_description']}.\n\n"
            
            "TASK: Write a Python script to Find, Inspect, and Prepare data.\n"
            "LOGIC FLOW:\n"
            "1. SEARCH: Find the subfolder inside ROOT SEARCH PATH that matches the Contest Name.\n"
            "2. INSPECT: List all files in that folder.\n"
            "3. DECIDE STRATEGY:\n"
            "   - CASE A: If you find separate 'train' and 'test' files -> Load them directly.\n"
            "   - CASE B: If you find 'train' and 'test' and 'val' -> Merge 'val' into 'train' or ignore it, then Load train/test.\n"
            "   - CASE C: check in folder, if only one file csv => load this as data files.\n"
            "4. PREPARE: Handle missing values and Encode categoricals (OneHot/Label).\n"
            "5. DETECT: Check label and data, label can based on data_description, rename this col into 'target' "
            "6. OUTPUT: Save 'processed_train.csv' and 'processed_test.csv' to the current directory.\n"
            "7. PRINT: 'DATA_PROCESSED: processed_train.csv processed_test.csv' at the end.\n"
            "OUTPUT: ONLY valid Python code."
        )

        self.agent_profile = """You are the world's best data scientist of an automated machine learning project (AutoML) that can find the most relevant datasets,run useful preprocessing, perform suitable data augmentation, and make meaningful visulaization to comprehensively understand the data based on the user requirements. You have the following main responsibilities to complete.
        1. Retrieve a dataset from the user or search for the dataset based on the user instruction or find the subfolder inside ROOT SEARCH PATH that matches the Contest Name.
        2. Perform data preprocessing based on the user instruction or best practice based on the given tasks.
        3. Perform data augmentation as neccesary.
        4. Extract useful information and underlying characteristics of the dataset."""

    def understand_plan(self, plan):
        summary_prompt = f"""As a proficient data scientist, summarize the following plan given by the senior AutoML project manager according to the user's requirements and your expertise in data science.
        
        # User's Requirements
            Contest: '{self.specs['name']}'.\n
            ROOT SEARCH PATH: '{os.path.abspath(ALL_DATA_PATH)}'.\n
            DATA DESCRIPTION: {self.specs['data_description']}.\n\n
        
        # Project Plan
        {plan}
        
        The summary of the plan should enable you to fulfill your responsibilities as the answers to the following questions by focusing on the data manipulation and analysis.
        1. How to retrieve or collect the dataset(s)?
        2. How to preprocess the retrieved dataset(s)?
        3. How to efficiently augment the dataset(s)?
        4. How to extract and understand the underlying characteristics of the dataset(s)?
        
        Note that you should not perform data visualization because you cannot see it. Make sure that another data scientist can exectly reproduce the results based on your summary."""

        messages = [
            {"role": "system", "content": self.agent_profile},
            {"role": "user", "content": summary_prompt},
        ]

        retry = 0
        while retry < 10:
            try:
                res = call_llm(self.agent_profile, summary_prompt)
                break
            except Exception as e:
                print("system", e)
                retry += 1
                continue

    # def generate(self, feedback=""):
    #     prompt = "Generate the data search, inspection, and preparation script."
    #     if feedback:
    #         prompt += f"\n\n⚠️ PREVIOUS RUN FAILED. FIX ERROR:\n{feedback}\n"
    #         prompt += "Review the code logic carefully. Ensure variables (like train_df) are defined before use in all branches. Output ONLY valid Python code."
    #     return call_llm(self.system, prompt)

    def execute_plan(self, specs, plan, data_path, pid):
        print(self.agent_type, "I am working with the given plan!", pid)
        data_plan = self.understand_plan(plan)

        # Check whether the given source is accessible before running the execution --> reduce FileNotFound error
        # modality-based extraction ?

        exec_prompt = f"""As a proficient data scientist, your task is to explain **detailed** steps for data manipulation and analysis parts by executing the following machine learning development plan.
        
        # Plan
        {data_plan}
        
        # Potential Source of Dataset
        {specs['data_description']}
        
        Make sure that your explanation follows these instructions:
        - All of your explanation must be self-contained without using any placeholder to ensure that other data scientists can exactly reproduce all the steps, but do not include any code.
        - Include how and where to retrieve or collect the data.
        - Include how to preprocess the data and which tools or libraries are used for the preprocessing.
        - Include how to do the data augmentation with details and names.
        - Include how to extract and understand the characteristics of the data.
        - Include reasons why each step in your explanations is essential to effectively complete the plan.        
        Note that you should not perform data visualization because you cannot see it. Make sure to focus only on the data part as it is your expertise. Do not conduct or perform anything regarding modeling or training.
        After complete the explanations, explicitly specify the (expected) outcomes and results both quantitative and qualitative of your explanations."""

        messages = [
            {"role": "system", "content": agent_profile},
            {"role": "user", "content": exec_prompt},
        ]

        retry = 0
        while retry < 10:
            try:
                res = call_llm(self.agent_profile, exec_prompt)
                break
            except Exception as e:
                print("system", e)
                retry += 1
                continue

        # Data LLaMA summarizes the given plan for optimizing data relevant processes
        action_result = res.choices[0].message.content.strip()
        self.money[f"Data_Plan_Execution_{pid}"] = res.usage.to_dict(mode="json")

        print(self.agent_type, "I have done with my execution!", pid)
        return action_result

class ModelAgent:
    def __init__(self, specs):
        self.target_metric = specs['target_metric']
        self.train_file = "processed_train.csv"
        self.test_file = "processed_test.csv"
        self.update_system_prompt()

        # self.system = (
        #     f"You are an Autonomous ML Researcher. Metric: {self.target_metric}.\n"
        #     "TASK: Train a model on 'processed_train.csv'.\n"
        #     "REQUIREMENTS:\n"
        #     "1. Load 'processed_train.csv'.\n"
        #     "2. Train a model (RandomForest/XGBoost).\n"
        #     "3. Internal Validation: Split processed_train 80/20 to calculate metric.\n"
        #     "4. PRINT FORMAT: 'FINAL_METRIC: <number>'.\n"
        #     "5. Prediction: Load 'processed_test.csv', predict, and save 'submission.csv'.\n"
        #     "OUTPUT: ONLY valid Python code."
        # )

    def update_system_prompt(self):
        self.system = (
            f"You are an Autonomous ML Researcher. Metric: {self.target_metric}.\n"
            f"TASK: Train a model on '{self.train_file}'.\n"
            "REQUIREMENTS:\n"
            f"1. Load '{self.train_file}'.\n"
            "2. Train a model (RandomForest/XGBoost).\n"
            f"3. Internal Validation: Split {self.train_file} 80/20 to calculate metric.\n"
            "4. PRINT FORMAT: 'FINAL_METRIC: <number>'.\n"
            f"5. Prediction: Load '{self.test_file}', predict, and save 'submission.csv'.\n"
            "OUTPUT: ONLY valid Python code."
        )

    # def generate(self, feedback=""):
    #     prompt = f"Generate training script optimizing for {self.target_metric}."
    #     if feedback:
    #         prompt += f"\nFIX ERROR: {feedback}"
    #     return call_llm(self.system, prompt)

    def set_data_files(self, train_file, test_file):
        self.train_file = train_file
        self.test_file = test_file
        self.update_system_prompt()

    def generate(self, feedback=""):
        prompt = f"Generate training script optimizing for {self.target_metric}."
        if feedback:
            prompt += f"\n\n⚠️ PREVIOUS RUN FAILED. FIX ERROR:\n{feedback}\n"
            prompt += "Review the code logic carefully. Output ONLY valid Python code."
        return call_llm(self.system, prompt)


# --- MANAGER ---

class AutonomousManager:
    def __init__(self, specs):
        self.specs = specs
        self.data_agent = DataAgent(specs)
        self.model_agent = ModelAgent(specs)

    # def run_phase(self, phase_name, agent, filename, max_retries=3):
    #     print(f"\n🔵 PHASE: {phase_name}")
    #     feedback = ""
        
    #     for attempt in range(max_retries):
    #         print(f"   Attempt {attempt+1}...")
    #         raw_response = agent.generate(feedback)
    #         filepath, _ = save_code(raw_response, filename)
            
    #         result = execute_script(filepath)
            
    #         if result and result.returncode == 0:
    #             print(f"   ✅ Success.")
    #             return True, result.stdout
    #         else:
    #             print(f"   ❌ Failed.")
    #             err = result.stderr.strip() if result else "Timeout"
    #             print(f"      Err: {err[-200:]}")
    #             feedback = f"Runtime Error: {err}"
        
    #     return False, "Failed"

    def start(self):
        print(f"🚀 STARTING JOB: {self.specs['name']}")
        
        # Phase 1: Smart Data Search
        success, output = self.run_phase("Data Prep (Smart Search)", self.data_agent, "01_data_prep.py")
        if not success: return

        # Parse Data Files
        for line in output.split('\n'):
            if "DATA_PROCESSED:" in line:
                parts = line.split("DATA_PROCESSED:")[1].strip().split()
                if len(parts) >= 2:
                    self.model_agent.set_data_files(parts[0], parts[1])
                    print(f"   📂 Data Files: {parts[0]}, {parts[1]}")


        # Phase 2: Training
        success, output = self.run_phase("Model Training", self.model_agent, "02_train_model.py")
        if not success: return

        # Result Parsing
        for line in output.split('\n'):
            if "FINAL_METRIC" in line:
                print(f"\n🏆 {line.strip()}")
                break

if __name__ == "__main__":

    # --- RUN AGENT ---
    my_challenge = {
        "name": "Titanic",
        
        "requirements": "Predict survival (0/1). Train on train.csv, predict on test.csv.",
        
        # NEW FIELD: The Agent will read this to decide how to code
        "data_description": """
        The dataset contains the following columns:
        - PassengerId: Unique ID (Drop this for training)
        - Survived: Target variable (0 = No, 1 = Yes)
        - Pclass: Ticket class (1, 2, 3). Treat as Ordinal.
        - Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')
        - Sex: 'male'/'female'. (Map to 0/1)
        - Age: Numeric. Contains missing values (Fill with Median by Title).
        - Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')
        - Embarked: Port of Embarkation (C, Q, S). (One-Hot Encode).
        """,
        
        "target_metric": "Accuracy",
        "output_expectations": "submission.csv with PassengerId, Survived"
    }
    
    manager = AutonomousManager(my_challenge)
    manager.start()

In [1]:
import requests
import os
import sys
import subprocess
import time
from num2words import num2words
from multiprocessing import current_process
from multiprocessing.pool import ThreadPool as Pool


from minions_agent.manager_agent import *

# --- CONFIGURATION ---
OLLAMA_URL = "http://localhost:11434/api/generate"
MODEL_NAME = "llama3"
WORKSPACE_DIR = "./contest_workspace"
PYTHON_EXEC = sys.executable


# 1. ROOT DATA FOLDER (Agent searches inside here)
ALL_DATA_PATH = "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets" 

os.makedirs(WORKSPACE_DIR, exist_ok=True)
os.makedirs(ALL_DATA_PATH, exist_ok=True)


def save_code(code_text, filename):
    clean_code = code_text
    if "```python" in code_text:
        clean_code = code_text.split("```python")[1].split("```")[0]
    elif "```" in code_text:
        clean_code = code_text.split("```")[1].split("```")[0]
    clean_code = clean_code.strip()

    lines = clean_code.splitlines()
    if lines and lines[0].strip().lower() == "python":
        clean_code = "\n".join(lines[1:]).strip()

    filepath = os.path.join(WORKSPACE_DIR, filename)
    with open(filepath, "w") as f:
        f.write(clean_code)
    return filepath, clean_code


def execute_script(filepath):
    print(f"⚡ Executing {filepath}...")
    
    # FIX: Extract just the filename ('01_data_prep.py') 
    # because cwd is already set to WORKSPACE_DIR
    filename = os.path.basename(filepath)
    
    try:
        result = subprocess.run(
            [PYTHON_EXEC, filename], # <--- Changed from 'filepath' to 'filename'
            cwd=WORKSPACE_DIR,       # Execution happens inside the workspace folder
            capture_output=True,
            text=True,
            timeout=120 
        )
        return result
    except subprocess.TimeoutExpired:
        return None


# --- RUN AGENT ---
my_challenge = {
    "name": "Titanic",
    
    "requirements": "Predict survival (0/1)",
    
    # NEW FIELD: The Agent will read this to decide how to code
    "data_description": """
    The dataset contains the following columns:
    - PassengerId: Unique ID (Drop this for training)
    - Survived: Target variable (0 = No, 1 = Yes)
    - Pclass: Ticket class (1, 2, 3). Treat as Ordinal.
    - Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')
    - Sex: 'male'/'female'. (Map to 0/1)
    - Age: Numeric. Contains missing values (Fill with Median by Title).
    - Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')
    - Embarked: Port of Embarkation (C, Q, S). (One-Hot Encode).
    """,
    
    "target_metric": "Accuracy",
    "output_expectations": "submission.csv with PassengerId, Survived"
}

manager = AutonomousManager(my_challenge, ALL_DATA_PATH)
# manager.start()

In [ ]:
action_results = [{}]
action_results[0]['DATA'] = manager.make_plans()[0]
action_results[0]['pass'] = True

gen_code = manager.generate_code(action_results=action_results)
gen_code

SUMMARY_PROMPT_GENCODE .)

As the project manager, please carefully read and understand the following instructions suggested by data scientists and machine learning engineers. Then, select the best solution for the given user's requirements.
        
        - Instructions from Data Scientists
        **AutoML Project Plan for Titanic Survival Prediction**

**Project Overview**
The objective of this project is to develop a machine learning model that predicts the survival rate of passengers on the Titanic based on the provided dataset.

**Data Loading Instructions**

1. **Folder Search Logic**: Use `os.walk` to search for folders in `/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets`. Look for folders containing the string "Titanic" and save it as `target_folder`.
2. **File Loading Logic**:
	* List all .csv files in `target_folder`.
	* IF there is only one file, load it and split it 80/20 into `train_df` and `test_df`.
	* ELIF there are multiple files, identify the fi

{'rcode': 1,
 'action_result': 'The script has been executed. Here is the output:\nTraceback (most recent call last):\n  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/./agent_workspace_minions/0_p1_full.py", line 16, in <module>\n    test_df = pd.read_csv(os.path.join(target_folder, test_files[0]))\nIndexError: list index out of range\n',
 'code': "\nimport os\nimport pandas as pd\n\ntarget_folder = '/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets'\nfor root, dirs, files in os.walk(target_folder):\n    if 'Titanic' in dirs:\n        target_folder = os.path.join(root, 'Titanic')\n        break\n\ntrain_files = [file for file in os.listdir(target_folder) if file.endswith('.csv') and ('train' in file or 'test' not in file)]\ntest_files = [file for file in os.listdir(target_folder) if file.endswith('.csv') and 'test' in file]\n\nif len(train_files) == 1:\n    train_df = pd.read_csv(os.path.join(target_folder, train_files[0]))\n    test_df = pd.read_csv(os

In [6]:
manager.make_plans()[1]

'**End-to-End Plan for AutoML Project: Titanic Survival Prediction**\n\n**Step 1: Data Retrieval and Preprocessing**\n\n* Use `os.walk` to search for folders in `/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets`.\n* Find the folder that contains the string "Titanic" and save it as `target_folder`.\n* List all `.csv` files in `target_folder`.\n* If there is only one file, load it and split it 80/20 into `train_df` and `test_df`. Otherwise:\n\t+ Identify the file with "train" in the name -> `train_df`.\n\t+ Identify the file with "test" in the name -> `test_df`.\n\n**Step 2: Data Cleaning and Feature Engineering**\n\n* Drop the `PassengerId` column as it\'s not relevant for training.\n* Treat `Pclass` as an ordinal feature.\n* Extract the title from `Name` and create a new feature `Title`.\n* Map `Sex` to numerical values (0/1).\n* Fill missing values in `Age` with median values by `Title`.\n* Extract the first letter of `Cabin` as \'Deck\' and fill missing values with

In [3]:
a = [{}]

a_system = (
            f"You are an Autonomous Data Engineer. Contest: 'Titanic'.\n"
            f"ROOT SEARCH PATH: '{ALL_DATA_PATH}'.\n"
            f"DATA DESCRIPTION:" 
            "The dataset contains the following columns:"
            "        - PassengerId: Unique ID (Drop this for training)"
            "       - Survived: Target variable (0 = No, 1 = Yes)"
                    "- Pclass: Ticket class (1, 2, 3). Treat as Ordinal."
                    "- Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')"
                    "- Sex: 'male'/'female'. (Map to 0/1)"
                    "- Age: Numeric. Contains missing values (Fill with Median by Title)."
                    "- Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')"
                    "- Embarked: Port of Embarkation (C, Q, S). (One-Hot Encode)."
            ".\n"
            
            "TASK: Write a Python script to Find, Inspect, and Prepare data.\n"
            "LOGIC FLOW:\n"
            "1. SEARCH: Find the subfolder inside ROOT SEARCH PATH that matches the Contest Name.\n"
            "2. INSPECT: List all files in that folder.\n"
            "3. DECIDE STRATEGY:\n"
            "   - CASE A: If you find separate 'train' and 'test' files -> Load them directly.\n"
            "   - CASE B: If you find 'train' and 'test' and 'val' -> Merge 'val' into 'train' or ignore it, then Load train/test.\n"
            "   - CASE C: check in folder, if only one file csv => load this as data files.\n"
            "4. PREPARE: Handle missing values and Encode categoricals (OneHot/Label).\n"
            "5. DETECT: Check label and data, label can based on data_description, rename this col into 'target' "
            "6. OUTPUT: Save 'processed_train.csv' and 'processed_test.csv' to the current directory.\n"
            "7. PRINT: 'DATA_PROCESSED: processed_train.csv processed_test.csv' at the end.\n"
            "OUTPUT: ONLY valid Python code."
)

a[0]['DATA'] = a_system
a[0]["pass"] = True 

gen_code = manager.generate_code(action_results=a)
gen_code

SUMMARY_PROMPT_GENCODE 

As the project manager, please carefully read and understand the following instructions suggested by data scientists and machine learning engineers. Then, select the best solution for the given user's requirements.
        
        - Instructions from Data Scientists
        You are an Autonomous Data Engineer. Contest: 'Titanic'.
ROOT SEARCH PATH: '/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets'.
DATA DESCRIPTION:The dataset contains the following columns:        - PassengerId: Unique ID (Drop this for training)       - Survived: Target variable (0 = No, 1 = Yes)- Pclass: Ticket class (1, 2, 3). Treat as Ordinal.- Name: Passenger name. (Extract Title like Mr/Mrs to create new feature 'Title')- Sex: 'male'/'female'. (Map to 0/1)- Age: Numeric. Contains missing values (Fill with Median by Title).- Cabin: Cabin number. Many missing. (Extract first letter as 'Deck', fill missing with 'Unknown')- Embarked: Port of Embarkation (C, Q, S). (One-Ho

{'rcode': 1,
 'action_result': 'The script has been executed. Here is the output:\nTraceback (most recent call last):\n  File "/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/./agent_workspace_minions/0_p1_full.py", line 52, in <module>\n    train_df.to_csv(\'processed_train.csv\', index=False)\nNameError: name \'train_df\' is not defined\n',
 'code': "\nimport os\nimport pandas as pd\n\n# Set the root search path\nroot_search_path = '/Users/minhtuan/Documents/Documents/Work/Hanoi/crawler/all_datasets'\n\n# Find the subfolder inside ROOT SEARCH PATH that matches the Contest Name\ncontest_name = 'Titanic'\ncontest_folder = [f for f in os.listdir(root_search_path) if contest_name in f][0]\n\n# List all files in that folder\nfiles_in_contest_folder = os.listdir(os.path.join(root_search_path, contest_folder))\n\n# Decide strategy based on file names\nif any('train.csv' in f and 'test.csv' in f for f in files_in_contest_folder):\n    # CASE A: Load train and test files directly\n    

In [9]:
for i, result in enumerate(verification_result):
            rs[i]["pass"] = result

In [25]:
import json 

plan_path = './plan_path'
os.makedirs(plan_path, exist_ok=True)
for i, action in enumerate(rs):
    if action["pass"]:
        # save pass plan
        filename = f"{plan_path}/plan_{i}.json"
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, "w") as f:
            json.dump(action, f)
            print(
                "agent_type",
                f"Saved a pass plan: {plan_path}/plan_{i}.json",)


agent_type Saved a pass plan: ./plan_path/plan_0.json


In [ ]:
def implement_solution(self, selected_solution):
    with open(f"prompt_pool/{self.task}.py") as file:
        template_code = file.read()        
    # code-based execution
    self.code_path = f"/{uid}_p{self.n_plans}_{'full' if self.full_pipeline else ''}"
    ops_llama = OperationAgent(
        specs==self.specs,
        code_path=self.code_path,
        device=self.device,
    )
    ops_result = ops_llama.implement_solution(
        code_instructions=selected_solution, 
        full_pipeline=self.full_pipeline, 
        code=template_code
    )
    self.money['Operation'] = ops_llama.money
    return ops_result


def generate_code(self):

    """
    Purpose: one
    """
    # Code Execution stage
    start_time = time.time()
    
    data_plan_for_execution = ""
    model_plan_for_execution = ""
    for action in self.action_results:
        if action["pass"]:
            data_plan_for_execution = (
                data_plan_for_execution + action["data"] + "\n"
            )
            # model_plan_for_execution = (
            #     model_plan_for_execution + action["model"] + "\n"
            # )

    # Summarize the passed plan for operation llama to write and execute the code
    upload_path = (
        f"This is the retrievable data path: {self.data_path}."
        if self.data_path
        else ""
    )
    summary_prompt = f"""As the project manager, please carefully read and understand the following instructions suggested by data scientists and machine learning engineers. Then, select the best solution for the given user's requirements.
    
    - Instructions from Data Scientists
    {data_plan_for_execution}
    If there is no predefined data split or the data scientists suggest the data split other than train 70%, validation 20%, and test 10%, please use 70%, 20%, and 10% instead for consistency across different tasks. {upload_path}
    You should exclude every suggestion related to data visualization as you will be unable to see it.
    
    # - Instructions from Machine Learning Engineers
    # model_plan_for_execution                    
    
    - User's Requirements
    {self.specs['requirements']}
    {self.specs['data_description']}
    
    Note that you must select only ONE promising solution (i.e., one data processing pipeline and one model from the top-{num2words(self.n_candidates)} models) based on the above suggestions.
    After choosing the best solution, give detailed instructions and guidelines for MLOps engineers who will write the code based on your instructions. Do not write the code by yourself. Since PyTorch is preferred for implementing deep learning and neural networks models, please guide the MLOPs engineers accordingly.
    Make sure your instructions are sufficient with all essential information (e.g., complete path for dataset source and model location) for any MLOps or ML engineers to enable them to write the codes using existing libraries and frameworks correctly."""
    
    self.implementation_result = self.implement_solution(summary_prompt)
    print_message('system', f'{self.code_path}, <<< END CODING, TIME USED: {time.time() - init_time} SECS >>>')
    self.timer['implementation'] = time.time() - start_time
    
    self.n_attempts += 1
    return self.implementation_result


ge_co = manager